## Analysis of Reduced System

In [1]:
import pyphi
import numpy as np
from pyphi import phi_spectrum

# Transition probability matrix used in Farnsworth
## Available as part of supplementary material: 
## https://www.sciencedirect.com/science/article/pii/S0303264720301969

tpm = np.array([
    [0.,0.,0.],
    [0.,0.,0.],
    [1.,0.,0.],
    [1.,0.,1.],
    [0.,1.,0.],
    [0.,1.,0.],
    [1.,1.,0.],
    [1.,1.,1.]
])

# Connectivity matrix for nodes (A,B,C)
# cm = np.array([
#     [0,1,1],
#     [1,0,1],
#     [1,1,0]
# ])

# Set up network object
network = pyphi.Network(tpm, node_labels=['R','C','P'])
print("Network = ",network.node_labels)

# Put the system into a given state
state = (1,1,1)
nodes = ['R','C','P']

## Get the requisite Subsystem
subsystem = pyphi.Subsystem(network, state, nodes)

## What does PyPhi Give? Should be 0.125
pyphi_val = pyphi.compute.phi(subsystem)
print("PyPhi Value = ",pyphi_val)

## Calculate all Phi values
display_CES= False  # if True, output will display constellations
Phi_Spectrum = phi_spectrum.get_phi_spectrum(subsystem,display_CES)

print("\nCuts = ",Phi_Spectrum[0])
print("\nPhi Spectrum = ",Phi_Spectrum[1])

Computing concepts:   0%|          | 0/7 [00:00<?, ?it/s]

Network =  NodeLabels(('R', 'C', 'P'))


PyPhi Value =  0.4375
	Number of Non-unique Constellations = 3

Evaluating Cut  Cut [R] ━━/ /━━➤ [C, P]
	Number of Non-unique Constellations = 1

Evaluating Cut  Cut [C] ━━/ /━━➤ [R, P]
	Number of Non-unique Constellations = 1

Evaluating Cut  Cut [R, C] ━━/ /━━➤ [P]
	Number of Non-unique Constellations = 1

Evaluating Cut  Cut [P] ━━/ /━━➤ [R, C]
	Number of Non-unique Constellations = 1

Evaluating Cut  Cut [R, P] ━━/ /━━➤ [C]
	Number of Non-unique Constellations = 1

Evaluating Cut  Cut [C, P] ━━/ /━━➤ [R]
	Number of Non-unique Constellations = 3

Cuts =  [Cut [R] ━━/ /━━➤ [C, P], Cut [C] ━━/ /━━➤ [R, P], Cut [R, C] ━━/ /━━➤ [P], Cut [P] ━━/ /━━➤ [R, C], Cut [R, P] ━━/ /━━➤ [C], Cut [C, P] ━━/ /━━➤ [R]]

Phi Spectrum =  [[0.6875, 0.1875, 0.4375], [0.6875, 1.187499, 0.9375], [0.6875, 0.9375], [1.0, 1.25], [1.0, 1.25], [0.625, 1.124999, 0.875, 0.874999]]


In [2]:
Phi_MIP = phi_spectrum.get_Phi_MIP(Phi_Spectrum)
print("Phi MIP = ",Phi_MIP)
print(len(Phi_MIP))
np.savetxt("results/phi_spectrum_farnsworth_reduced.csv", Phi_MIP, delimiter=",")
print("Results Saved")

Phi MIP =  [0.1875 0.4375 0.625  0.6875]
4
Results Saved


#### Now get all the different cause/effect repetoires

In [60]:
# network = pyphi.Network(tpm, cm, node_labels=['R','C','P'])

# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)

# Get the cause repetoire for the mechanism over the purview
R,C,P = subsystem.node_indices
# print(subsystem.state)
mechanism = (C,)
purview = (R,P)
cr = subsystem.effect_repertoire(mechanism,purview)
flat_cr = pyphi.distribution.flatten(cr)  # returns 1d array in little endian convention
print(flat_cr)

mip = subsystem.effect_mip(mechanism,purview)
print(mip)

[1. 0. 0. 0.]
Repertoire irreducibility analysis
  φ = 1/4
  Mechanism: [C]
  Purview = [R, P]
  Direction: EFFECT
  Partition:
     ∅     C 
    ─── ✕ ───
     P     R 
  Repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    1      │
    │ 10    0      │
    │ 01    0      │
    │ 11    0      │
    └──────────────┘
  Partitioned repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    3/4    │
    │ 10    0      │
    │ 01    1/4    │
    │ 11    0      │
    └──────────────┘


## Now do the same for the unidirectional cut R -//->CP

In [1]:
import pyphi
import numpy as np

# Transition probability matrix. Saying where each state (little-endian binary) goes
tpm = np.array([
    [1.,0.,0.,0.,0.,0.,0.,0.],
    [1.,0.,0.,0.,0.,0.,0.,0.],
    [0.,1/2.,0.,0.,0.,1/2.,0.,0.],
    [0.,1/2.,0.,0.,0.,1/2.,0.,0.],
    [0.,0.,1.,0.,0.,0.,0.,0.],
    [0.,0.,1.,0.,0.,0.,0.,0.],
    [0.,0,0.,1/2.,0.,0.,0.,1/2.],
    [0.,0,0.,1/2.,0.,0.,0.,1/2.],
])

# Set up network object
# network = pyphi.Network(tpm, cm=cm, node_labels=['A','B','C'])
network = pyphi.Network(tpm, node_labels=['R','C','P'])

# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)

# Get the cause repetoire for the mechanism over the purview
R,C,P = subsystem.node_indices
# print(subsystem.state)
mechanism = (C,)
purview = (R,P)
cr = subsystem.effect_repertoire(mechanism,purview)
flat_cr = pyphi.distribution.flatten(cr)  # returns 1d array in little endian convention
print(flat_cr)

mip = subsystem.effect_mip(mechanism,purview)
print(mip)


[1. 0. 0. 0.]
Repertoire irreducibility analysis
  φ = 1/4
  Mechanism: [C]
  Purview = [R, P]
  Direction: EFFECT
  Partition:
     ∅     C 
    ─── ✕ ───
     P     R 
  Repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    1      │
    │ 10    0      │
    │ 01    0      │
    │ 11    0      │
    └──────────────┘
  Partitioned repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    3/4    │
    │ 10    0      │
    │ 01    1/4    │
    │ 11    0      │
    └──────────────┘


In [7]:
import pyphi
import numpy as np
import itertools
from pyphi import Direction
from pyphi import utils
from pyphi import cache
from pyphi import models

# Transition probability matrix. Saying where each state (little-endian binary) goes
tpm = np.array([
    [1.,0.,0.,0.,0.,0.,0.,0.],
    [1.,0.,0.,0.,0.,0.,0.,0.],
    [0.,1/2.,0.,0.,0.,1/2.,0.,0.],
    [0.,1/2.,0.,0.,0.,1/2.,0.,0.],
    [0.,0.,1.,0.,0.,0.,0.,0.],
    [0.,0.,1.,0.,0.,0.,0.,0.],
    [0.,0,0.,1/2.,0.,0.,0.,1/2.],
    [0.,0,0.,1/2.,0.,0.,0.,1/2.],
])

# Set up network object
# network = pyphi.Network(tpm, cm=cm, node_labels=['A','B','C'])
network = pyphi.Network(tpm, node_labels=['R','C','P'])

# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)

## Find all concepts for the specified subsystem
all_concepts = []
mechanisms = utils.powerset(subsystem.node_indices, nonempty=True)
for mechanism in mechanisms:
#     print("MECHANISM = ",mechanism)
    mechanism_concepts = []
    
    # Find the CORE cause(s) for a given mechanism
    direction = Direction.CAUSE
    past_purviews = subsystem.potential_purviews(direction, mechanism, purviews=False)
    all_causes = [subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews]


    core_cause = max(all_causes)
    
#     print(core_cause)
    
    
    if core_cause.phi > 0.:
        all_core_causes = [value for index,value in enumerate(subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_cause.phi]
    else:
        all_core_causes = [core_cause]
            
#     print("Core Cause(s) for Mechanism : ",all_core_causes)
    
    # Find the CORE effect(s) for a given mechanism
    direction = Direction.EFFECT
    future_purviews = subsystem.potential_purviews(direction, mechanism, purviews=False)
    all_effects = [subsystem.find_mip(direction, mechanism, purview) for purview in future_purviews]


    core_effect = max(all_effects)
    
#     print(core_effect)
    
    
    if core_effect.phi > 0.:
        all_core_effects = [value for index,value in enumerate(subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_effect.phi]
    else:
        all_core_effects = [core_effect]
            
#     print("Core Effects(s) for Mechanism : ",all_core_effects)

    ## Now we need to build concepts for all possible core cause and core effect combinations
    for each_cause in all_core_causes:
        MIC = models.MaximallyIrreducibleCause(each_cause)
        for each_effect in all_core_effects:
            MIE = models.MaximallyIrreducibleEffect(each_effect)
            
            concept = models.Concept(mechanism = mechanism, cause = MIC, effect = MIE, subsystem = subsystem)            
                            
            ## If phi^max > 0 store the concept
            if concept.phi > 0.:
                mechanism_concepts.append(concept)
            
    if mechanism_concepts:
        all_concepts.append(mechanism_concepts)
        
print(all_concepts)
        

# Create all possible CES via the cartesian product of all possible concepts for each mechanism
original_CES = []
for element in itertools.product(*all_concepts):
    original_CES.append(models.CauseEffectStructure(concepts=element))
    
for CES in original_CES:
    print("Possible CES:")
    print(CES)

[[━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
           Concept: Mechanism = [C], φ = 1/2            
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            MIC                         MIE             
┌──────────────────────────┐┌──────────────────────────┐
│  φ = 1/2                 ││  φ = 1/2                 │
│  Purview = [P]           ││  Purview = [R]           │
│  MIP:                    ││  MIP:                    │
│     ∅     C              ││     ∅     C              │
│    ─── ✕ ───             ││    ─── ✕ ───             │
│     P     ∅              ││     R     ∅              │
│  Repertoire:             ││  Repertoire:             │
│    ┌─────────────┐       ││    ┌─────────────┐       │
│    │ S    Pr(S)  │       ││    │ S    Pr(S)  │       │
│    │ ╴╴╴╴╴╴╴╴╴╴╴ │       ││    │ ╴╴╴╴╴╴╴╴╴╴╴ │       │
│    │ 0    1      │       ││    │ 0    1      │       │
│    │ 1    0      │       ││    │ 1    0      │       │
│    └─────────────┘       ││